## Imports 

In [1]:
import pandas as pd 
import numpy as np 

from scipy import stats
import math 

from mplsoccer import PyPizza, add_image, FontManager
import matplotlib.pyplot as plt 

## Retrieve data including 30 of the best wingers in Europe

In [3]:
df = pd.read_csv("../../data/top_forwards_europe/stats_per90.csv")

In [8]:
df.shape

(30, 46)

## Clean the data

In [5]:
df['Player'] = df['Player'].str.split('\\',expand=True)[0]

In [6]:
df.head()

,Player,Span,Pos,Squad,MP,Starts,Min,90s,Gls,Ast,...,PassDead.1,Drib.1,Touches,Mid 3rd,Att 3rd,Att Pen,Succ%,TotDist.1,PrgDist.1,CPA
0,Son Heung-min,2020-21,FW,Tottenham,37,36,3114,34.6,0.49,0.29,...,0.06,0.06,44.5,19.0,23.5,4.13,62.3,164.9,85.0,0.95
1,Sadio Man�,2020-21,FW,Liverpool,35,31,2810,31.2,0.35,0.22,...,0.00,0.00,51.0,19.4,32.6,9.07,59.2,219.0,123.3,1.73
2,Cristiano Ronaldo,2020-21,FW,Juventus,33,31,2802,31.1,0.93,0.06,...,0.00,0.10,47.0,21.7,27.2,6.85,64.4,207.1,134.7,1.90
3,Mohamed Salah,2020-21,FW,Liverpool,37,34,3078,34.2,0.64,0.15,...,0.00,0.00,51.4,18.2,33.9,9.15,50.5,184.7,100.9,1.58
4,Kylian Mbapp�,2020-21,FW,Paris S-G,31,27,2380,26.4,1.02,0.26,...,0.00,0.27,54.6,18.7,38.7,10.50,46.6,241.9,163.4,2.61


## Load some fonts

In [9]:
font_normal = FontManager(("https://github.com/google/fonts/blob/main/apache/roboto/static/"
                           "Roboto-Regular.ttf?raw=true"))
font_italic = FontManager(("https://github.com/google/fonts/blob/main/apache/roboto/static/"
                           "Roboto-Italic.ttf?raw=true"))
font_bold = FontManager(("https://github.com/google/fonts/blob/main/apache/roboto/static/"
                         "Roboto-Medium.ttf?raw=true"))

## Select some columns

In [11]:
df.columns

Index(['Player', 'Span', 'Pos', 'Squad', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G+A', 'G-PK', 'G+A-PK', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90',
       'G/Sh', 'G/SoT', 'PassesCmp', 'PassesAtt', 'PassesCmp%', 'TotDist',
       'PrgDist', 'KP', '1/3', 'PPA', 'CrsPA', 'Prog', 'SCA', 'PassLive',
       'PassDead', 'Drib', 'GCA', 'PassLive.1', 'PassDead.1', 'Drib.1',
       'Touches', 'Mid 3rd', 'Att 3rd', 'Att Pen', 'Succ%', 'TotDist.1',
       'PrgDist.1', 'CPA'],
      dtype='object')

In [ ]:
params = ["Gls",'SoT','SoT%', 'G/Sh',"Ast",'PassesCom%','KP', 'SCA', 'GCA', 'Touches', 'Succ%']